<a href="https://colab.research.google.com/github/jegadeesh2001/NLP-Tablets-Annotation/blob/main/OCR_TRAINED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
 
with open('/content/ocr_data_annotations.json', 'r') as f:
    data = json.load(f)
    
print(data['examples'][0])

{'content': "vsasnesciencostintet Amingaon.Norh Guwahati CautonI is dangerous to take ths preparabon exoept under Medical Supervision. WARNINGTo be sold by on t prescription of a Regislered Medical Caution  is dangeroos to take this KEEPTHEMEDICINCOUTOFREACH OF CHILDREN Composition: Each uncoatea tablel.contain Metformin Hyctochlonde IP Gliclazide IP Exciplents ColourLake Quinaine Yellow Dosage: As directed by the Physician Storage:Store between 10C to 25C at dry&dark place KEEPTHE MEDICINEOUTOF REACH OF CHILDREN Y Composition Gliclazide Metformin Hydrochloride Tablets CybexM40 Gliclazide Metformin Hydrochloride Tablets CyblexM40  40 0 CyblexM 500g 40mg SCHEDULE'H'DRUG q.s Practisoner oniy Amingaon.Norh Guwahati Dist.Kamrup-781031(Assam) Vig.Lic.No.377/DRMg/2014 Manufactured by SLifesciences Limited Amingaon.North Guwahati Dist.Kamrup-781031(Assam Caution  is dangeroos to take this", 'seen_by': [{'annotator': '19pd15@psgtech.ac.in', 'annotator_id': 1}], 'comments': [], 'metadata': {'':

In [ ]:
training_data = {'classes' : ["Name", "Components","Manufacturer"], 'annotations' : []}
for example in data['examples']:
  temp_dict = {}
  temp_dict['text'] = example['content']
  temp_dict['entities'] = []
  for annotation in example['annotations']:
    start = annotation['start']
    end = annotation['end']
    label = annotation['tag'].upper()
    temp_dict['entities'].append((start, end, label))
  training_data['annotations'].append(temp_dict)
  
print(training_data['annotations'][0])

{'text': "vsasnesciencostintet Amingaon.Norh Guwahati CautonI is dangerous to take ths preparabon exoept under Medical Supervision. WARNINGTo be sold by on t prescription of a Regislered Medical Caution  is dangeroos to take this KEEPTHEMEDICINCOUTOFREACH OF CHILDREN Composition: Each uncoatea tablel.contain Metformin Hyctochlonde IP Gliclazide IP Exciplents ColourLake Quinaine Yellow Dosage: As directed by the Physician Storage:Store between 10C to 25C at dry&dark place KEEPTHE MEDICINEOUTOF REACH OF CHILDREN Y Composition Gliclazide Metformin Hydrochloride Tablets CybexM40 Gliclazide Metformin Hydrochloride Tablets CyblexM40  40 0 CyblexM 500g 40mg SCHEDULE'H'DRUG q.s Practisoner oniy Amingaon.Norh Guwahati Dist.Kamrup-781031(Assam) Vig.Lic.No.377/DRMg/2014 Manufactured by SLifesciences Limited Amingaon.North Guwahati Dist.Kamrup-781031(Assam Caution  is dangeroos to take this", 'entities': [(520, 571, 'NAME'), (340, 350, 'COMPONENT'), (300, 325, 'COMPONENT'), (326, 339, 'COMPONENT')

In [ ]:
training_data

{'classes': ['Name', 'Components', 'Manufacturer'],
 'annotations': [{'text': "vsasnesciencostintet Amingaon.Norh Guwahati CautonI is dangerous to take ths preparabon exoept under Medical Supervision. WARNINGTo be sold by on t prescription of a Regislered Medical Caution  is dangeroos to take this KEEPTHEMEDICINCOUTOFREACH OF CHILDREN Composition: Each uncoatea tablel.contain Metformin Hyctochlonde IP Gliclazide IP Exciplents ColourLake Quinaine Yellow Dosage: As directed by the Physician Storage:Store between 10C to 25C at dry&dark place KEEPTHE MEDICINEOUTOF REACH OF CHILDREN Y Composition Gliclazide Metformin Hydrochloride Tablets CybexM40 Gliclazide Metformin Hydrochloride Tablets CyblexM40  40 0 CyblexM 500g 40mg SCHEDULE'H'DRUG q.s Practisoner oniy Amingaon.Norh Guwahati Dist.Kamrup-781031(Assam) Vig.Lic.No.377/DRMg/2014 Manufactured by SLifesciences Limited Amingaon.North Guwahati Dist.Kamrup-781031(Assam Caution  is dangeroos to take this",
   'entities': [(520, 571, 'NAME'),
 

In [ ]:
len(training_data)

2

In [ ]:
!python -m spacy download en_core_web_lg -q

In [ ]:
!pip install spacy[transformers]

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin() # create a DocBin object

In [ ]:
from spacy.util import filter_spans

for training_example  in tqdm(training_data['annotations']): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy")

100%|██████████| 33/33 [00:00<00:00, 239.88it/s]

Skipping entity
Skipping entity


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0

2023-02-12 15:41:11.192244: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-12 15:41:11.192381: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-12 15:41:11.192406: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-02-12 15:41:20,369] [INFO] Set up n

In [ ]:
!pip install 'spacy[transformers]'

In [ ]:
!zip -r ner.zip '/content/model-best'

In [ ]:
import spacy_transformers

In [ ]:
nlp_ner = spacy.load("model-best")

In [ ]:
doc=nlp_ner("Each film coated tablet contains you must not take this medicine if you are preonant or may likely become pregnant during treatment. Isotretinoin IP (micronized 10 mg Excipients q.s. anixa Colour:Ponceau 4R Lake Dosage: LifoScionces As directed by the Physician A WHO GMP Certified Company Storage:Store below 25C Manufactured by Protect from sunlight and moisture Canixa Life Sciences Pvt.Ltd KEEP OUT OF REACH OF CHILDREN K.No.313.Raipur Industrial Area.Bhagwanpur See pack insert before use Roorkee,Dist. Haridwar.Uarakhand-247667 Email:info@canixalife.com Mfg.Lic.No.:3/UA/SC/P-2016 TM-Trademark Under RegistraSon Saferet-10 Saferet-10 Saferet-10 sotretinin (micronied)Tablets 10mg A Saferet-10 TM SCHEOULE H PRESCRIPTION DRUG-CAUTION Not to be sold by retail without the prescription of a egitered Medical Practitioner 14-10 WARNING TO FEMALE PATIENTS Composition : This medicine may cause severe birth dedects you must not take this medicine if y ou are pregnant Each film coated tablet contains or may likely become pregnant during treatmant Isotretinoin IP (micronized10 mg Excipients q.s. anixa ColourPonceau 4R Lake Dosage: Lile Scioncos As directed by the Physician (A WHO GMP Certified Company) Storage:Store below 25C Manufactured by: Drntant trom cunlinht and mnichara Canixa Life Sciences Pvt.Ltd")

In [ ]:
colors = {"COMPONENT": "#F67DE3", "NAME": "#7DF6D9", "MANUFACTURER":"#FFFFFF"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [ ]:
import spacy
import pickle

# Load your model


# Save the model to a pickle file
with open("/content/drive/MyDrive/spacy_model.pkl", "wb") as f:
    pickle.dump(nlp_ner, f)

In [ ]:
import pickle

# Load the saved model from the pickle file
with open("spacy_model.pkl", "rb") as f:
    nlp = pickle.load(f)

In [ ]:
doc=nlp("Each film coated tablet contains you must not take this medicine if you are preonant or may likely become pregnant during treatment. Isotretinoin IP (micronized 10 mg Excipients q.s. anixa Colour:Ponceau 4R Lake Dosage: LifoScionces As directed by the Physician A WHO GMP Certified Company Storage:Store below 25C Manufactured by Protect from sunlight and moisture Canixa Life Sciences Pvt.Ltd KEEP OUT OF REACH OF CHILDREN K.No.313.Raipur Industrial Area.Bhagwanpur See pack insert before use Roorkee,Dist. Haridwar.Uarakhand-247667 Email:info@canixalife.com Mfg.Lic.No.:3/UA/SC/P-2016 TM-Trademark Under RegistraSon Saferet-10 Saferet-10 Saferet-10 sotretinin (micronied)Tablets 10mg A Saferet-10 TM SCHEOULE H PRESCRIPTION DRUG-CAUTION Not to be sold by retail without the prescription of a egitered Medical Practitioner 14-10 WARNING TO FEMALE PATIENTS Composition : This medicine may cause severe birth dedects you must not take this medicine if y ou are pregnant Each film coated tablet contains or may likely become pregnant during treatmant Isotretinoin IP (micronized10 mg Excipients q.s. anixa ColourPonceau 4R Lake Dosage: Lile Scioncos As directed by the Physician (A WHO GMP Certified Company) Storage:Store below 25C Manufactured by: Drntant trom cunlinht and mnichara Canixa Life Sciences Pvt.Ltd")

In [ ]:
doc1=nlp("aber each film coated tablet contain : isotretinoin ip ( micronized ) 10 mg excipient qs . colour : ponceau 4r lake d : life sciences eel ) as directed by the physician . commend p from sunlight and cist ° canal lis selectee part ltd. no . 313 , rabipur industrial area , baganda , ore mp , lc no . :3uascp-2016 ti agen ye nope , saferet10 saferet10 saferet '' 10 * « isotretinoin ( micronized ) tablets 10 mg saferet-10 wre-10 warning to female patient composition : this medicine may cause severe b")

In [ ]:
colors = {"COMPONENT": "#F67DE3", "NAME": "#7DF6D9", "MANUFACTURER":"#FFFFFF"}
options = {"colors": colors} 

spacy.displacy.render(doc1, style="ent", options= options, jupyter=True)

In [ ]:
entities_1 = [(ent.text,ent.label_) for ent in doc1.ents]

In [ ]:
entities_1

[('Metadoxine 500mg&Siymann 140mg', 'COMPONENT'),
 ('Metadoxine 140mg', 'COMPONENT'),
 ('Silymarin Livapt Mg', 'MANUFACTURER')]